## SQL ##

Este es el codigo usado en sql para crear nuestra base de datos.

In [ ]:
"""
CREATE DATABASE IF NOT EXIST AIRBB
USE AIRBB

CREATE A TABLE Hosting (
urls VARCHAR(255)
record_id INT,
Titles VARCHAR(255),
Property_types VARCHAR (255),
Host_name VARCHAR (255),
PRIMARY KEY (Ids),
);

CREATE A TABLE Description_ (
id (INT)
Host_name VARCHAR (255),
Prices_per_night INT,
Check_ins DATETIME,
Check_outs DATETIME,
Cleaning_fees INT,
Maximum_guests INT,
Location VARCHAR (255),
Beds INT,
Baths INT,
Rooms INT,
FOREING KEY (id) REFERENCE Hosting (record_id)

);

CREATE A TABLE Ratings (
record_id INT,
Ratings INT,
Num_reviews VARCHAR(255),
FOREING KEY (record_id) REFERENCE Hosting (record_id)
);
"""

Codigo para crear desde python la base de datos en sql 

In [ ]:
db = mysql.connector.connect(host     = "localhost",
                             user     = "root",
                             password = "password",
                             database = database)

cursor = db.cursor()

cursor.execute('CREATE DATABASE IF NOT EXIST AIRBB') #Con el cursor lo que estamos haciendo básicamente es usar las queris aqui pero alli, no se explicarme mejor sorry.

cursor.execute('USE AIRBB')

cursor.execute('''CREATE A TABLE IF NOT EXIST Hosting (
                    urls VARCHAR(255)
                    record_id INT,
                    Titles VARCHAR(255),
                    Property_types VARCHAR (255),
                    Host_name VARCHAR (255),
                    PRIMARY KEY (record_id),
                        );''' )

cursor.execute('''CREATE A TABLE IF NOT EXIST Description_ (
                    id (INT)
                    Host_name VARCHAR (255),
                    Prices_per_night INT,
                    Check_ins DATETIME,
                    Check_outs DATETIME,
                    Cleaning_fees INT,
                    Maximum_guests INT,
                    Location VARCHAR (255),
                    Camas INT,
                    Baños INT,
                    Dormitorios INT,
                    FOREING KEY (id) REFERENCE Hosting (record_id)
                );''' )

cursor.execute('''CREATE A TABLE IF NOT EXIST Ratings (
                    record_id INT,
                    Ratings INT,
                    Num_reviews INT,
                    FOREING KEY (record_id) REFERENCE Hosting (record_id)
                    ); ''')

cursor.fetchall() # Vaciamos el cursor
cursor.close()
db.close()

Este es el codigo para la subida a la base de datos

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('df_clean.csv')
df.head()

Dividimos las columnas del df en distintos df segun las tablas de nuestra base de datos.

In [ ]:
df_Hosting = df[['urls','record_id','Titles','Property_types','Host_name']]

In [ ]:
df_Description_ = df[['record_id','Host_name', 'Prices_per_night','Check_ins','Check_outs','Cleaning_fees','Maximum_guests','Location','Camas','Baños','Dormitorios']]
df_Description_ = df_Description_.rename(columns={'record_id': 'id'})


In [ ]:
df_Ratings = df[['record_id','Ratings','Num_reviews']]

Aqui nos aseguramos de que no haya ningun nan sustituyendolos todos por none (valor que si admite sql).

In [ ]:
df_Description_ = df_Description_.replace({np.nan : None}) #Sustituimos todos los posibles nan por none para poder subirlos
df_Hosting = df_Hosting.replace({np.nan : None})
df_Ratings = df_Ratings.replace({np.nan : None})

Aqui procedemos a la subida de las distintas tablas.

Hosting

In [ ]:
column_names= ['urls','record_id','Titles','Property_types','Host_name']

In [ ]:
database = "AIRBB"
table_name = "Hosting"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="password",
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_HOSTING.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_HOSTING[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Description_

In [ ]:
column_names = ['id','Host_name', 'Prices_per_night','Check_ins','Check_outs','Cleaning_fees','Maximum_guests','Location','Camas','Baños','Dormitorios']

In [ ]:
database = "AIRBB"
table_name = "Description_"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="password",
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_DESCRIPTION_.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_DESCRIPTION_[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Ratings

In [ ]:
column_names= ['record_id','Ratings','Num_reviews']

In [ ]:
database = "AIRBB"
table_name = "Ratings"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="password",
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_HOSTING.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_HOSTING[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

# Extracción de datos

In [ ]:
import mysql.connector

In [ ]:
def extraer_datos_mysql(host, user, password, database, query):
    # Conectarse a la base de datos
    db = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    
    # Crear cursor para ejecutar la consulta
    cursor = db.cursor()

    # Ejecutar la consulta
    cursor.execute(query)
    
    # Obtener los nombres de las columnas
    column_names = cursor.column_names
    print("Nombres de las columnas:", column_names)

    # Guardar y mostrar los datos de la tabla
    data = cursor.fetchall()
    for fila in data:
        print(fila)

    # Cerrar el cursor y la conexión
    cursor.close()
    db.close()


Hosting

In [ ]:
host = "localhost"
user = "root"
password = "password"
database = "AIRBB"
query = "SELECT * FROM Hosting;"

extraer_datos_mysql(host, user, password, database, query)

Description_

In [ ]:
host = "localhost"
user = "root"
password = "password"
database = "AIRBB"
query = "SELECT * FROM Description_;"

extraer_datos_mysql(host, user, password, database, query)

Ratings

In [ ]:
host = "localhost"
user = "root"
password = "password"
database = "AIRBB"
query = "SELECT * FROM Ratings;"

extraer_datos_mysql(host, user, password, database, query)